In [1]:
require 'nn';
require 'cudnn';
require 'cunn';
require 'struct'
require 'image';

In [2]:
function traverse(writerParam, writerAct, layer, input)
    local layerName = torch.typename(layer)
    local output
    if layer.modules then
        if layerName == 'nn.Sequential' then
            output = input
            for i=1, #layer.modules do
                output = traverse(writerParam, writerAct, layer.modules[i], output)
            end
        elseif layerName == 'nn.ConcatTable' then
            output = {}
            for i=1, #layer.modules do
                table.insert(output, traverse(writerParam, writerAct, layer.modules[i], input))
            end
        else
            assert('Unknown container ' .. layerName)
        end
    else
        output = layer:forward(input)
        if layerName ~= 'nn.Identity'then
            params = {}
            counter = 0
            if layer.running_mean and layer.running_var then
                table.insert(params, layer.running_mean)
                table.insert(params, layer.running_var)
            end
            if layer.weight and layer.bias then
                table.insert(params, layer.weight)
                table.insert(params, layer.bias)
            end
            for i=1, #params do
                param = params[i]
                counter = counter + param:nElement()
                for i=1, param:nElement() do
                    writerParam:write(struct.pack('<f', param:view(-1)[i]))
                end
            end
            if counter > 0 then
                print(("%s params: write %d bytes"):format(layerName, counter))
            end
            
            writerAct:write(("%s %d %d %d\n"):format(layerName, output:size()[2], output:size()[3], output:size()[4]))
            for i=1,output:nElement() do
                writerAct:write(output:view(-1)[i])
                writerAct:write("\t")
            end
            writerAct:write("\n")
            print(("%s act: write %d bytes"):format(layerName, output:nElement()))
        end
    end
    return output
end 

In [3]:
data = torch.load('input.t7'):float()
modelCPU = cudnn.convert(torch.load('/home/minint/modelzoo/resnet152/model.t7'), nn):float()
modelCPU:evaluate()
torch.manualSeed(11)

In [ ]:
image = data[1]:view(1, 3, 224, 224)
writerParam = assert(io.open('sixiao/param.bin', 'wb'))
writerAct = assert(io.open('sixiao/act.bin', 'wb'))
traverse(writerParam, writerAct, modelCPU, image)
writerParam:close()
writerAct:close()

nn.SpatialConvolution params: write 9472 bytes	


nn.SpatialConvolution act: write 802816 bytes	


nn.SpatialBatchNormalization params: write 256 bytes	


nn.SpatialBatchNormalization act: write 802816 bytes	


nn.ReLU act: write 802816 bytes	


...

nn.CAddTable act: write 802816 bytes	


nn.ReLU act: write 802816 bytes	


nn.SpatialConvolution params: write 16448 bytes	


nn.SpatialConvolution act: write 200704 bytes	


nn.SpatialBatchNormalization params: write 256 bytes	
